# World View Viz-1

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mp
import sklearn as sk
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

World View - World map visualization with covid cases per country.

In [1]:
# url = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# url = 'https://raw.githubusercontent.com/uw-covid-hackathon/covid-visualization/master/OXFORD_latest_imputed.csv'
url = 'https://raw.githubusercontent.com/uw-covid-hackathon/covid-visualization/master/OXFORD_latest_daily_columns.csv'

In [3]:
latest_csv = pd.read_csv(url, error_bad_lines=False)
# df = latest_csv.dropna(subset=['GovernmentResponseIndex'])
latest_csv = latest_csv.drop(columns = ['M1_Wildcard'])
policies = ['C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief',  'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'M1_Wildcard']

Focus on one Date for now, March 25th

In [4]:
# df = df[df['GovernmentResponseIndex']>0]
# df_us = df[df['CountryName'] == 'United States']
# df_us['ConfirmedCases'].diff()
df_march_25 = latest_csv[latest_csv['Date'] == '2020-04-25']
df_march_25

,CountryName,Date,CountryCode,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,...,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay,NewConfirmedCases,NewConfirmedDeaths
115,Afghanistan,2020-04-25,AFG,3.0,1.0,3.0,0.0,2.0,1.0,4.0,...,83.33,83.33,64.74,64.74,76.52,76.52,0.0,0.0,70.0,1.0
279,Albania,2020-04-25,ALB,3.0,1.0,2.0,1.0,2.0,1.0,4.0,...,91.67,91.67,80.77,80.77,84.09,84.09,62.5,62.5,15.0,0.0
443,Algeria,2020-04-25,DZA,3.0,1.0,2.0,1.0,2.0,1.0,4.0,...,91.67,91.67,65.71,65.71,70.83,70.83,37.5,37.5,120.0,8.0
607,Andorra,2020-04-25,AND,3.0,1.0,2.0,1.0,1.0,1.0,0.0,...,76.19,76.19,55.77,55.77,56.82,56.82,50.0,50.0,7.0,3.0
771,Angola,2020-04-25,AGO,3.0,1.0,2.0,1.0,2.0,1.0,4.0,...,83.33,83.33,67.31,67.31,70.45,70.45,50.0,50.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28088,Venezuela,2020-04-25,VEN,3.0,1.0,3.0,1.0,2.0,1.0,3.0,...,85.71,85.71,67.31,67.31,70.45,70.45,50.0,50.0,20.0,0.0
28252,Vietnam,2020-04-25,VNM,3.0,0.0,2.0,1.0,2.0,1.0,3.0,...,83.33,83.33,69.23,69.23,77.27,77.27,25.0,25.0,0.0,0.0
28416,Yemen,2020-04-25,YEM,3.0,1.0,1.0,0.0,2.0,1.0,0.0,...,55.95,55.95,28.21,28.21,33.33,33.33,0.0,0.0,0.0,0.0
28580,Zambia,2020-04-25,ZMB,3.0,1.0,0.0,NaN,2.0,0.0,4.0,...,63.81,63.81,59.29,59.29,65.53,65.53,25.0,25.0,8.0,0.0


Choropleth colors

Columns with mouse hover text for each country


In [5]:
scl = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], 
       [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#4d0000']] # reds

df_map = df_march_25[['CountryCode','CountryName', 'Date', 'NewConfirmedCases', 'NewConfirmedDeaths']]
print(df_map.info())
for col in df_map.columns:
    df_map[col] = df_map[col].astype(str)

df_map['text'] = df_map['CountryName']+' \nCases: ' + df_map['NewConfirmedCases'] +' \nDeaths: '+df_map['NewConfirmedDeaths']


<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 115 to 28744
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CountryCode         176 non-null    object 
 1   CountryName         176 non-null    object 
 2   Date                176 non-null    object 
 3   NewConfirmedCases   176 non-null    float64
 4   NewConfirmedDeaths  176 non-null    float64
dtypes: float64(2), object(3)
memory usage: 8.2+ KB
None


In [6]:
df_map

,CountryCode,CountryName,Date,NewConfirmedCases,NewConfirmedDeaths,text
115,AFG,Afghanistan,2020-04-25,70.0,1.0,Afghanistan \nCases: 70.0 \nDeaths: 1.0
279,ALB,Albania,2020-04-25,15.0,0.0,Albania \nCases: 15.0 \nDeaths: 0.0
443,DZA,Algeria,2020-04-25,120.0,8.0,Algeria \nCases: 120.0 \nDeaths: 8.0
607,AND,Andorra,2020-04-25,7.0,3.0,Andorra \nCases: 7.0 \nDeaths: 3.0
771,AGO,Angola,2020-04-25,0.0,0.0,Angola \nCases: 0.0 \nDeaths: 0.0
...,...,...,...,...,...,...
28088,VEN,Venezuela,2020-04-25,20.0,0.0,Venezuela \nCases: 20.0 \nDeaths: 0.0
28252,VNM,Vietnam,2020-04-25,0.0,0.0,Vietnam \nCases: 0.0 \nDeaths: 0.0
28416,YEM,Yemen,2020-04-25,0.0,0.0,Yemen \nCases: 0.0 \nDeaths: 0.0
28580,ZMB,Zambia,2020-04-25,8.0,0.0,Zambia \nCases: 8.0 \nDeaths: 0.0


Creating the data object for the plotting.

In [7]:
fig = go.Figure(data=go.Choropleth(
    locations = df_map['CountryCode'],
    z = df_map['NewConfirmedCases'].astype(float),
    text = df_map['text'],
    colorscale = 'Picnic',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '-',
    colorbar_title = 'Covid Cases',
))

fig.update_layout(
    title_text='COVID - March -25th',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: Oxford Data',
        showarrow = False
    )]
)


Going for World Map

In [8]:
url = 'https://raw.githubusercontent.com/uw-covid-hackathon/covid-visualization/master/OXFORD_latest_daily_columns.csv'
latest_csv = pd.read_csv(url, error_bad_lines=False)

In [9]:
data_slider = []
for date in latest_csv.Date.unique():

    # I select the year (and remove DC for now)
    df_date = latest_csv[latest_csv['Date'] == date]

    for col in df_date.columns:  # I transform the columns into string type so I can:
        df_date[col] = df_date[col].astype(str)

    ### I create the text for mouse-hover for each state, for the current year    
    df_date['text'] = df_date['CountryName']+' Cases: ' + df_date['NewConfirmedCases'] +' Deaths: '+ df_date['NewConfirmedDeaths']

    ### create the dictionary with the data for the current year
    data_per_date =dict(
        type='choropleth',
        locations = df_date['CountryCode'],
        z = df_date['NewConfirmedCases'].astype(float),
        text = df_date['text'],
        colorscale = 'Picnic',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_tickprefix = '-',
        colorbar_title = 'Covid Cases',
    )

    data_slider.append(data_per_date)  # I add the dictionary to the list of dictionaries for the slider


In [10]:
steps = []
dates = latest_csv.Date.unique()
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='{}'.format(dates[i])) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]  

In [12]:
layout = dict(title_text='COVID CASES per DAY',geo=dict(showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'),
        sliders=sliders)
fig = dict(data=data_slider, layout=layout) 
plotly.offline.plot(fig)

'temp-plot.html'